# Analysieren von Kassenbelegen mit der Formularerkennung

![Ein Roboter, der einen Kassenbeleg hält](./images/receipt_analysis.jpg)

Maschinelles Sehen gehört zur Künstlichen Intelligenz (KI) und wird häufig eingesetzt, um mit optischer Zeichenerkennung (Optical Character Recognition, OCR) gedruckten oder handschriftlichen Text zu lesen. Dazu wird der Text aus den Dokumenten oft in ein Format extrahiert, das zur weiteren Verarbeitung oder Analyse verwendet werden kann.

In komplexeren OCR-Szenarien können Informationen aus Formularen wie etwa Bestellungen oder Rechnungen extrahiert werden, wobei mithilfe einer semantischen Analyse die Bedeutung der Felder interpretiert werden kann. Die **Formularerkennung** wurde speziell für diese Art von KI-Problemen entwickelt.

## Anzeigen eines Kassenbelegs

In diesem Beispiel verwenden Sie das integrierte Modell der Formularerkennung zur Analyse von Kassenbelegen.

Klicken Sie links neben der Zelle auf die Schaltfläche **Zelle ausführen** (&#9655;), um die Zelle auszuführen und ein Beispiel für einen Kassenbeleg anzuzeigen, den Sie mit der Formularerkennung analysieren.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# Load and display a receipt image
fig = plt.figure(figsize=(6, 6))
image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Erstellen einer Formularerkennungsressource

Erstellen Sie zunächst eine Formularerkennungsressource in Ihrem Azure-Abonnement:

1. Öffnen Sie das Azure-Portal unter „https://portal.azure.com“ in einer neuen Browserregisterkarte, und melden Sie sich mit Ihrem Microsoft-Konto an.
2. Klicken Sie auf **+ Ressource erstellen**, und suchen Sie nach *Formularerkennung*.
3. Wählen Sie in der Liste der Dienste **Formularerkennung** aus.
4. Wählen Sie im Blatt **Formularerkennung** die Option **Erstellen** aus.
5. Geben Sie auf dem Blatt **Erstellen** die folgenden Informationen ein, und wählen Sie dann **Erstellen** aus.
   * **Name**: Ein eindeutiger Name für Ihren Dienst
   * **Abonnement**: Ihr Azure-Abonnement
   * **Region**: Eine beliebige verfügbare Region
   * **Tarif**: F0
   * **Ressourcengruppe**: Die zuvor verwendete vorhandene Ressourcengruppe
   * **Ich bestätige, dass ich den folgenden Hinweis gelesen und verstanden habe**: Ausgewählt
6. Warten Sie, bis der Dienst erstellt ist.
7. Öffnen Sie Ihren neu erstellten Formularerkennungsdienst im Azure-Portal, kopieren Sie auf der Seite **Schlüssel und Endpunkt** den **Schlüssel1** und **Endpunkt** für Ihre Ressource, und fügen Sie die Werte in die folgende Codezelle anstelle von **YOUR_FORM_KEY** und **YOUR_FORM_ENDPOINT** ein.

In [ ]:
form_key = 'YOUR_FORM_KEY'
form_endpoint = 'YOUR_FORM_ENDPOINT'

print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

## Analysieren eines Kassenbelegs

Jetzt können Sie die Formularerkennung verwenden, um einen Kassenbeleg zu analysieren.

In [ ]:
import os
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

# Create a client for the form recognizer service
form_recognizer_client = FormRecognizerClient(endpoint=form_endpoint, credential=AzureKeyCredential(form_key))

try:
    print("Analyzing receipt...")
    # Get the receipt image file
    image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')

    # Submit the file data to form recognizer
    with open(image_path, "rb") as f:
        analyze_receipt = form_recognizer_client.begin_recognize_receipts(receipt=f)
    
    # Get the results
    receipt_data = analyze_receipt.result()

    # Print the extracted data for the first (and only) receipt
    receipt = receipt_data[0]
    receipt_type = receipt.fields.get("ReceiptType")
    if receipt_type:
        print("Receipt Type: {}".format(receipt_type.value))
    merchant_address = receipt.fields.get("MerchantAddress")
    if merchant_address:
        print("Merchant Address: {}".format(merchant_address.value))
    merchant_phone = receipt.fields.get("MerchantPhoneNumber")
    if merchant_phone:
        print("Merchant Phone: {}".format(merchant_phone.value))
    transaction_date = receipt.fields.get("TransactionDate")
    if transaction_date:
        print("Transaction Date: {}".format(transaction_date.value))
    print("Receipt items:")
    items = receipt.fields.get("Items")
    if items:
        for idx, item in enumerate(receipt.fields.get("Items").value):
            print("\tItem #{}".format(idx+1))
            item_name = item.value.get("Name")
            if item_name:
                print("\t - Name: {}".format(item_name.value))
            item_total_price = item.value.get("TotalPrice")
            if item_total_price:
                print("\t - Price: {}".format(item_total_price.value))
    subtotal = receipt.fields.get("Subtotal")
    if subtotal:
        print("Subtotal: {} ".format(subtotal.value))
    tax = receipt.fields.get("Tax")
    if tax:
        print("Tax: {}".format(tax.value))
    total = receipt.fields.get("Total")
    if total:
        print("Total: {}".format(total.value))

except Exception as ex:
    print('Error:', ex)


Mit der Formularerkennung können Sie außerdem Daten im Formular interpretieren, um beispielsweise die Adresse und Telefonnummer des Händlers, das Datum der Transaktion sowie die einzelnen Positionen, Zwischensummen, Steuern und Endbeträge zu identifizieren.

## Weitere Informationen

Weitere Informationen zur Formularerkennung finden Sie in der [Dokumentation zur Formularerkennung](https://docs.microsoft.com/de-de/azure/cognitive-services/form-recognizer/index).